In [1]:
import numpy as np
import hashlib

In [2]:
def read_input(fname):
    rawdata = []
    with open(fname, 'r') as inf:
        for line in inf.readlines():
            if line.strip() == '':
                continue
            rawdata.append([c for c in line.strip()])

    rawdata = np.array(rawdata)
    data = np.zeros(rawdata.shape, dtype=int)
    data[np.where(rawdata == '#')] = 2
    data[np.where(rawdata == 'O')] = 1
    return data

In [3]:
def tilt_north(data):
    # Tilt north
    tilted = np.zeros(data.shape, dtype=int)
    tilted[np.where(data==2)] = 2
    ny, nx = data.shape
    for col in range(0, nx):
        # print('Col', col)
        # print(data[:, col])
        cubes = np.where(data[:, col] == 2)[0]
        # print('Cubes', cubes)
        if len(cubes):
            n_stones = sum(data[:, col][:cubes[0]])
            # print('B', n_stones)
            tilted[:, col][0:n_stones] = 1
            for i, c in enumerate(cubes[:-1]):
                n_stones = sum(data[:, col][c+1:cubes[i+1]])
                # print('M', i, c, n_stones, c+1, c+n_stones)
                # print(tilted[:, col])
                tilted[:, col][c+1:c+n_stones+1] = 1
                # print(tilted[:, col])

            n_stones = sum(data[:, col][cubes[-1]+1:])
            c = cubes[-1]
            # print('E', i, c, n_stones, c+1, c+n_stones)
            tilted[:, col][c+1:c+n_stones+1] = 1
        else:
            n_stones = sum(data[:, col])
            # print('N', n_stones)

            tilted[:, col][0:n_stones] = 1

    return tilted

def tilt_west(data):
    # Tilt west
    tilted = np.zeros(data.shape, dtype=int)
    tilted[np.where(data==2)] = 2
    ny, nx = data.shape
    for row in range(0, ny):
        # print('Col', col)
        # print(data[:, col])
        cubes = np.where(data[row, :] == 2)[0]
        # print('Cubes', cubes)
        if len(cubes):
            n_stones = sum(data[row, :][:cubes[0]])
            # print('B', n_stones)
            tilted[row, :][0:n_stones] = 1
            for i, c in enumerate(cubes[:-1]):
                n_stones = sum(data[row, :][c+1:cubes[i+1]])
                # print('M', i, c, n_stones, c+1, c+n_stones)
                # print(tilted[:, col])
                tilted[row, :][c+1:c+n_stones+1] = 1
                # print(tilted[:, col])

            n_stones = sum(data[row, :][cubes[-1]+1:])
            c = cubes[-1]
            # print('E', i, c, n_stones, c+1, c+n_stones)
            tilted[row, :][c+1:c+n_stones+1] = 1
        else:
            n_stones = sum(data[row, :])
            # print('N', n_stones)
            tilted[row, :][0:n_stones] = 1

    return tilted

def calc_load(data):
    load = 0
    ny, nx = data.shape
    for row in range(0, ny):
        load += np.where(data[row, :] == 1)[0].size * (ny-row)
 
    return load

def cycle(data):
    tilted = tilt_north(data)
    tilted = tilt_west(tilted)
    tilted = tilt_north(tilted[::-1, :])[::-1, :]
    tilted = tilt_west(tilted[:, ::-1])[:, ::-1]

    return tilted

def find_load(data):
    loads = []
    load = 0
    count = 1
    recur = -1
    start = 0
    while count < 100000:
        data = cycle(data)
        data_hash = hashlib.sha256(np.ascontiguousarray(data)).hexdigest()
        load = calc_load(data)
        if (data_hash, load) in loads and recur == -1:
            start = loads.index((data_hash, load))
            recur = count - start -1
            break
        loads.append((data_hash, load))
        count += 1

    rest = int((1000000000-start) % recur)
    return loads[start+rest-1][1]

In [4]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

data = read_input('input14a.txt')

load = calc_load(tilt_north(data))
    
print(f'Load is {load}')

assert load == 136

print('\nPuzzle case\n')

data = read_input('input14.txt')

load = calc_load(tilt_north(data))
    
print(f'Load is {load}')

assert load == 110090

print('\n*****\nPuzzle2\n*****\n')

data = read_input('input14a.txt')

load = find_load(data)

print(f'Load after 1000000000 cycles is {load}')

assert load == 64 

print('\nPuzzle case\n')

print('Test case\n')

data = read_input('input14.txt')

load = find_load(data)

print(f'Load after 1000000000 cycles is {load}')

assert load == 95254


*****
Puzzle1
*****

Test case

Load is 136

Puzzle case

Load is 110090

*****
Puzzle2
*****

Load after 1000000000 cycles is 64

Puzzle case

Test case

Load after 1000000000 cycles is 95254
